### CS4423 - Networks
Prof. Götz Pfeiffer<br />
School of Mathematics, Statistics and Applied Mathematics<br />
NUI Galway

# Lecture 8: Graph Isomorphism and Symmetries

Two graphs $G = (X, E)$ and $H = (Y, F)$ are said to be **isomorphic** if there
is an edge preserving bijection between their vertex sets $X$ and $Y$.

[Deciding graph isomorphism](https://en.wikipedia.org/wiki/Graph_isomorphism_problem)
is computationally hard.

An isomorphism of a graph $G$ with itself is called an **automorphism**,
or a symmetry of $G$.

We will apply BFS to the problem of computing the **group** of all
automorphisms of a given tree.

Symmetries, or the lack thereof, are interesting properties of networks ...

In random selections, for example of trees on $n$ vertices, it turns out that
more symmetric species are less frequently picked ... 

In [ ]:
import networkx as nx
opts = { "with_labels": True, "node_color": 'y'}

**Example.** According to Cayley's formula, there are $4^{4-2} = 16$
trees on $n = 4$ vertices.  But overall, we only see $2$
distinct structures, a **path graph** of length $3$, and a **star graph**
with $3$ spikes.

These structures are known as **unlabelled trees**
(as opposed to a **labelled tree**, where each node corresponds to
a specific element of $\{0, \dots, n{-}1\}$).

As a random graph, the path graph occurs far more often than
the star graph.  Is something wrong with the assumption of uniform
distribution?

In [ ]:
n = 4
T = nx.random_tree(n)
nx.draw(T)

No, there isn't.  It's just that the line **shape** appears more often than the
star **shape** in the full list of all **labelled** graphs on 4 points.

![4-trees](images/t4.png)

## Graph Isomorphism

If we are mainly interested in the network **structure**
of a graph $G = (X, E)$, the underlying vertex set $X$ is replacable.
The following definition makes this notion precise.

<div class="alert alert-danger">

<b>Definition.</b>  
    <ul>
        <li> 
            Let $G_1 = (X_1, E_2)$ 
            and $G_2 = (X_2, E_2)$ be graphs.
        </li>
        <li>
            A **graph isomorphism** from $G_1$ to $G_2$
            is a **bijective** map $f \colon X_1 \to X_2$
            such that $(f(x), f(y))$ is an edge of $G_2$
            if and only if $(x, y) \in E_1$.
        </li>
        <li>
            We say that $G_1$ is isomorphic to $G_2$
            (and write $G_1 \cong G_2$) if
            there is a graph isomorphism $f$ from $G_1$ to $G_2$.
        </li>
        <li>
            In the special case where both $X_1$ and $X_2$
            are the same set $X$,
            a bijection $f \colon X \to X$ is called
            a **permutation** of $X$.
        </li>
        <li>
            If a permutation $f$ of $X$ is a graph isomorphism
            from $G_1 = (X, E_1)$ to $G_2 = (X, E_2)$
            then in fact both $E_1$ and $E_2$ are the same edge set $E$
            and $f$ is called a **graph automorphism** of the
            graph $G = (X, E)$.
        </li>
    </ul>
</div>

<div class="alert alert-success">

**Examples.** 
    
* Consider the bijection $f(0) = A$, $f(1) = B$, $f(2) = C$,
    or $f = (^0_A{}^1_B{}^2_C)$ for short.
    
* $f$ is a graph isomorphism
    from a graph $G$: $0 - 1 - 2$ to a graph $H$: $A - B - C$.

* But not from $G$ to $K: C - A - B$: the $f$-image of the edge
    $1 - 2$ in $G$ is not an edge in $K$.
    
* The permutation $(^{0\,1\,2}_{2\,1\,0})$
    is a graph automorphism of $G$.
    
* And so is the **identity permutation** $(^{0\,1\,2}_{0\,1\,2})$.
    
* But not the permutation $(^{0\,1\,2}_{1\,0\,2})$.
    
</div>

* Any bijection $f \colon X \to Y$ can be used to produce an **isomorphic copy** of a graph $G = (X, E)$.

* It takes some care to decide **equality** of graphs.
(Apparently, `networkx` does not have a function for that ...)

In [ ]:
G = nx.Graph([(1,2)])
H = nx.Graph([(2,1)])

In [ ]:
print(list(G.edges()), list(H.edges()))

In [ ]:
def normalized_edges(G):
    return tuple(sorted(tuple(set(e)) for e in G.edges()))

In [ ]:
def are_equal_as_graphs(G, H):
    if set(G.nodes()) != set(H.nodes()):
        return false
    return normalized_edges(G) == normalized_edges(H)

In [ ]:
are_equal_as_graphs(G, H)

In [ ]:
T = nx.random_tree(n)
nx.draw(T, **opts)

*  **Note:** the automorphisms of a graph form a **group**.

<div class="alert alert-success">

Recall that a **group** is a set $S$ together with a special **identity** element $e \in S$, an **inversion** map $i \colon S \to S$ and a **multiplication** map
$m \colon S \times S \to S$ such that
* $m(m(a,b),c) = m(a, m(b, c))$ for all $a, b, c \in S$: multiplication is associative;
* $m(a, i(a)) = m(i(a), a) = e$ for all $a \in S$;
* $m(e, a) = m(a, e) = a$ for all $a \in S$.
</div>

**Example:** The **symmetric group** $S_n$ of all permutations of the $n$-set
$X = \{0, 1, 2, \dots, n-1\}$.

<div class="alert alert-danger">

**Definition**.  A **group action** of a group $S$ on a set $X$
is a map $S \times X \to X$, mapping $(s, x) \mapsto s(x)$,
such that
* $e(x) = e$ for all $x \in X$;
* $b(a(x)) = m(b, a)(x)$ for all $a, b \in S$, and all $x \in X$.
</div>

**Example:** $S_n$ acts on $X = \{0, \dots, n-1\}$.

If a group $S$ acts on a set $X$ then to each point $x \in X$ are associated
* its **orbit** $S(x) = \{s(x) : s \in S\} \subseteq X$;
* its **stablizer** $S_x = \{ s \in S : s(x) = x \} \subseteq S$.

The stablizer $S_x$ is a **subgroup** of $S$.

<div class="alert alert-success">

**Orbit-Stablizer Theorem.**  $|S_x| \cdot |S(x)| = |S|$ for all $x \in X$.
</div>

* $S_n$ acts on trees with vertex set $X$  by **relabelling** the nodes.
* $S_n$ is generated by the **transpositions** of 
  consecutive numbers: $k \leftrightarrow k+1$.
* This generating set defines a **graph** on the set of all trees on $X$.
* The connected component (unlabelled graph) of a tree `T` can
  be constructed by BFS as the **orbit** of `T` under the $S_n$-action.
* As a by-product, the **automorphism group** of is determined
as the **stabilizer** of `T`.

## Permutations as Tuples

*  A permutation $f$ of $X = \{0, \dots, n-1\}$
can be represented as a python tuple `t` of length $n$,
with the understanding that `t[i]` represents
$f(i)$ for $i\ in X$.

* **Don't confuse `python` tuples with the usual cycle notation for permutations!!**

* Here, the tuples are **image lists**.

* The **identity** permutation $f(x) = x$:

In [ ]:
def identity(n):
    return tuple(k for k in range(n))

In [ ]:
one = identity(n)
one

* The **transposition** $i \leftrightarrow i{+}1$:

In [ ]:
def transposition(n, i):
    t = [k for k in range(n)]
    t[i], t[i+1] = t[i+1], t[i]
    return tuple(t)

In [ ]:
t1 = transposition(n, 1)
t2 = transposition(n, 2)
print(t1, t2)

* **Composition** of `dict`s:  $(g \circ f)(x) = g(f(x))$.

In [ ]:
def composition(a, b):
    return tuple(a[k] for k in b)

* $\bigl({}^{0123}_{0213}\bigr) \circ \bigl({}^{0123}_{0132}\bigr) 
= \bigl({}^{0123}_{0231}\bigr)$,


* $\bigl({}^{0123}_{0132}\bigr) \circ \bigl({}^{0123}_{0213}\bigr)
= \bigl({}^{0123}_{0312}\bigr)$.

In [ ]:
t12 = composition(t1, t2)
t12

In [ ]:
t21 = composition(t2, t1)
t21

* **Inversion:** $f^{-1}(y) = x \iff f(x) = y$.

In [ ]:
def inverse(a):
    b = list(a)
    for k, v in enumerate(a):
        b[v] = k
    return tuple(b)

* $\bigl({}^{0123}_{0312}\bigr)^{-1}
= \bigl({}^{0123}_{0231}\bigr)$

In [ ]:
inverse(t21)

In [ ]:
inverse(t21) == t12

* The **symmetric group** $S_n$ of all permutations of $X = \{0, 1, \dots, n-1\}$ is generated by the transpositions $(i, i{+}1)$, $i = 0, \dots, n-2$.

In [ ]:
gens = [transposition(n, k) for k in range(n-1)]
gens

* **BFS** can enumerate the orbit of a point $x$ under a list of generators of the acting group.
* This variant of BFS is called the **orbit algorithm** in Computational Group Theory.
* The `python` function `orbit` takes as its arguments:
    - a list `gens` of generators of a group $S$
    - the point `x` from $X$
    - a function `action` describing the action $S \times X \to X$
    - and a function `equals` for testing equality in $X$.

* In many cases, the standard `python` equals operator will work as equality test:

In [ ]:
def eq(x, y):
    return x == y

* The action of permutations as tuples is simply indexing:

In [ ]:
def apply(a, x):
    return a[x]

* With `eq` as an example for the `equals` parameter, and `apply` for `action` in mind,
  here is the orbit algorithm as a loop (over `gens`) within a loop (over the growing orbit `xxx`).

In [ ]:
def orbit(gens, x, action, equals):
    xxx = [x]
    for y in xxx:
        for a in gens:
            z = action(a, y)
            w = next((v for v in xxx if equals(v, z)), None)
            if w is None:
                xxx.append(z)
    return xxx

* For example, the orbit of a single point $x \in \{0, \dots, n-1\}$
under the action of the symmetric group $S_n$ (which of course is all of $X$)
can now be computed as follows:

In [ ]:
orbit(gens, 1, apply, eq)

* Or, regarding composition $S \times S \to S$ as an action of $S$ on itself
(check the definition!), one can enumerate the $n!$ elements of $S_n$:

In [ ]:
elements = orbit(gens, one, composition, eq)
len(elements)

In [ ]:
print(elements)

## Orbits of Isomorphic Trees

* The process of **relabelling** the nodes of a graph with vertex set
  $X$ by permutations of $X$ defines an **action** of $S_n$
  on the set of all labelled trees on $X$.

In [ ]:
def relabel(a, G):
    mapping = dict(enumerate(a))
    return nx.relabel_nodes(G, mapping)

* $1_X(T) = T$ and $(g \circ f)(T) = g(f(T))$ for each tree $T$
  with vertex set $X$.

In [ ]:
G = nx.path_graph(n)
G12 = relabel(t2, relabel(t1, G))
print(list(G12.nodes()))
GG = relabel(t21, G)
print(list(GG.nodes()))

*  Using `relabel` as action, and `are_equal_as_graphs` as equality check,
the above orbit algorithm finds the 
orbit of a tree $T$ under $S_n$,
i.e., the list of **all trees** on the vertex set $X$ that are **isomorphic** to $T$.

In [ ]:
nx.draw(T, **opts)

In [ ]:
TTT = orbit(gens, T, relabel, are_equal_as_graphs)

In [ ]:
len(TTT)

**Graph and Queue version:**

* Implicitly, the action of a group $S$ on a set $X$, with respect to
a set $A$ of generators of $S$, defines an **action graph** with vertex
set $X$, and edges $(x, y)$ whenever there is an element $a \in A$
with $a(x) = y$.
* Here, we need to supply and manage an explicit queue for the BFS. 

In [ ]:
from queue import Queue

* We turn the simultaneous initialization of the queus and the graph into a small subroutine.

In [ ]:
def init_Q_G(x):
    Q = Queue()
    Q.put(x)
    G = nx.Graph()
    G.add_node(x)
    return Q, G

* `action_graph` is a variant of the above `orbit` function 
that has the same set of parameters, and computes and returns the orbit
of $x$ as a graph.

In [ ]:
def action_graph(gens, x, action, equals):
    Q, G = init_Q_G(x)
    while not Q.empty():
        y = Q.get()
        for a in gens:
            z = action(a, y)
            w = next((v for v in G.nodes() if equals(v, z)), None)
            if w is None:
                Q.put(z)
                G.add_node(z)
                G.add_edge(y, z)
            else:
                G.add_edge(y, w)
    return G

* For example, the graph of the action of $S_n$ on $X = \{0, \dots, n-1\}$.

In [ ]:
G = action_graph(gens, 1, apply, eq)
nx.draw(G, **opts)

* For example, the action of $S_n$ on its own elements

In [ ]:
G = action_graph(gens, one, composition, eq)
nx.draw(G, **opts)

* For example, the action of $S_n$ on trees with vertex set $X$

In [ ]:
G = action_graph(gens, T, relabel, are_equal_as_graphs)
nx.draw(G, **opts)

![4-trees](images/t4-action.png)

**Transversal tree.**

* A **transversal** is a list of group elements $f_y$, one for each $y$
in the orbit of $x$ such that $f_y(x) = y$ (and $f_x = e$).

* The next variant of BFS computes such a transversal in the form of `"perm"`
attributes of the nodes of a **spanning tree** of the action graph.

* This essentially uses the group to measure the **distance** from $x$.

In [ ]:
def transversal_tree(gens, x, action, equals):
    Q, G = init_Q_G(x)
    G.nodes[x]['perm'] = identity(len(gens[0]))
    while not Q.empty():
        y = Q.get()
        for a in gens:
            z = action(a, y)
            w = next((v for v in G.nodes() if equals(v, z)), None)
            if w is None:
                Q.put(z)
                G.add_node(z)
                G.add_edge(y, z)
                G.nodes[z]['perm'] = composition(a, G.nodes[y]['perm'])
    return G

In [ ]:
G = transversal_tree(gens, 1, apply, eq)
nx.get_node_attributes(G, 'perm')

In [ ]:
G = transversal_tree(gens, one, composition, eq)
nx.get_node_attributes(G, 'perm')

In [ ]:
G = transversal_tree(gens, T, relabel, are_equal_as_graphs)
nx.get_node_attributes(G, 'perm')

**Automorphisms.**

* The stabilizer $S(x)$ of a point $x \in X$ is generated by loops in
the action graph.

* More precisely, there is the following theorem, aka 
[Schreier's subgroup lemma](https://en.wikipedia.org/wiki/Schreier%27s_lemma).

<div class="alert alert-danger">

**Theorem.**
Suppose a group $S$, generated by a set $A$,
acts on a set $X$ and that $\{f_y : y \in S(x)\}$ is a transversal
of the orbit of $x \in X$.  Then
$$
\{f_{a(y)}^{-1} \circ a \circ f_y: a \in A,\, y \in S(x)\}
$$
is a set of generators for the stabilizer $S_x$.
</div>

In [ ]:
def automorphisms(gens, x, action, equals):
    Q, G = init_Q_G(x)
    G.nodes[x]['perm'] = identity(len(gens[0]))
    autos = set()
    while not Q.empty():
        y = Q.get()
        for a in gens:
            z = action(a, y)
            w = next((v for v in G.nodes() if equals(v, z)), None)
            if w is None:
                Q.put(z)
                G.add_node(z)
                G.add_edge(y, z)
                G.nodes[z]['perm'] = composition(a, G.nodes[y]['perm'])
            else:
                ay = composition(a, G.nodes[y]['perm'])
                loop = composition(inverse(G.nodes[w]['perm']), ay)
                autos.add(loop)
    return list(autos)

For example, $S_n$ acting on $\{0,\dots,n{-}1\}$:

In [ ]:
aut = automorphisms(gens, 1, apply, eq)
aut

In [ ]:
orbit(aut, one, composition, eq)

For example, $S_n$ acting by composition (multiplication) on itself:

In [ ]:
aut = automorphisms(gens, one, composition, eq)
aut

For example, $S_n$ acting on $n$-trees:

In [ ]:
aut = automorphisms(gens, T, relabel, are_equal_as_graphs)
aut

In [ ]:
orbit(aut, one, composition, eq)

In [ ]:
n = 6
gens = [transposition(n, k) for k in range(n-1)]

In [ ]:
T = nx.random_tree(n)
nx.draw(T, **opts)

In [ ]:
aut = automorphisms(gens, T, relabel, are_equal_as_graphs)
aut

In [ ]:
orbit(aut, identity(n), composition, eq)

##  Code Corner

### `python`

* `sorted` [doc]
* `tuple` [doc]
* `set` [doc]
* `next` [doc]
* `[].pop` [doc]
* `{}.keys` [doc]
* `{}.values` [doc]
* `{}.items` [doc]

### `networkx`

* `relabel_nodes`
* `path_graph`

## Exercises

1.  How many unlabelled trees are there on $n = 5$ vertices?
   What (sizes) are their automorphism groups?

2. $n = 6$?